In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import split_scale
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt

In [2]:
url = get_db_url('zillow')
query = ('''
SELECT id, bathroomcnt AS bathroom, bedroomcnt AS bedroom, 
    calculatedfinishedsquarefeet AS square_feet, 
    regionidcounty AS county, taxvaluedollarcnt AS house_value 
FROM properties_2017
''')
df_orig = pd.read_sql(query, url)

In [3]:
df = df_orig.copy()
df.head(10)

,id,bathroom,bedroom,square_feet,county,house_value
0,0,0.0,0.0,NaN,3101.0,9.0
1,1,0.0,0.0,NaN,3101.0,27516.0
2,2,0.0,0.0,73026.0,3101.0,1434941.0
3,3,0.0,0.0,5068.0,3101.0,1174475.0
4,4,0.0,0.0,1776.0,3101.0,440101.0
5,5,0.0,0.0,2400.0,3101.0,287634.0
6,6,0.0,0.0,NaN,3101.0,563029.0
7,7,0.0,0.0,3611.0,3101.0,698984.0
8,8,0.0,0.0,NaN,3101.0,9.0
9,9,0.0,0.0,3754.0,3101.0,265184.0


In [4]:
df.isnull().sum()

id                 0
bathroom        2957
bedroom         2945
square_feet    45097
county          2932
house_value    34266
dtype: int64

In [5]:
df.shape[0]

2985217

In [6]:
bathroom_std = df.bathroom.std()
bedroom_std = df.bedroom.std()
square_feet_std = df.square_feet.std()
house_value_std = df.house_value.std()

df.bedroom = df.bedroom[(df.bedroom <= bedroom_std * 10)]
df.bathroom = df.bathroom[(df.bathroom <= bathroom_std * 10)]
df.square_feet = df.square_feet[(df.square_feet <= 
                                         square_feet_std *10)]
df.house_value = df.house_value[
    (df.house_value <= house_value_std * 10) & 
    (df.house_value >= house_value_std * .01)]

In [7]:
ave_bedrooms = round(df.bedroom.mean())
df.bedroom.replace(0, ave_bedrooms, inplace=True)

ave_bathrooms = round(df.bathroom.mean())
df.bathroom.replace(0, ave_bathrooms, inplace=True)

In [8]:
df.isnull().sum()

id                 0
bathroom        3605
bedroom         3207
square_feet    46067
county          2932
house_value    48360
dtype: int64

In [9]:
df.dropna(inplace=True)
df.shape[0]

2906141

In [10]:
df.county = df.county.astype(int)
df.square_feet = df.square_feet.astype(int)
df.bedroom = df.bedroom.astype(int)

In [11]:
df.nlargest(215, 'house_value')

,id,bathroom,bedroom,square_feet,county,house_value
2338477,2338477,7.0,4,7675,1286,8160012.0
534375,534375,5.0,3,6956,1286,8158697.0
1352789,1352789,5.5,5,4443,1286,8158697.0
1879041,1879041,9.0,7,10350,3101,8158693.0
1499297,1499297,7.0,5,5626,3101,8157278.0
...,...,...,...,...,...,...
2070865,2070865,9.0,7,13483,1286,7830216.0
1162654,1162654,8.0,6,10971,3101,7830000.0
1071178,1071178,4.5,4,4865,1286,7822823.0
885676,885676,7.0,5,7070,3101,7817425.0


In [15]:
train, test = split_scale.split_my_data(df, .8)
train.head()

,id,bathroom,bedroom,square_feet,county,house_value
308504,308504,1.5,3,1468,1286,190858.0
1622796,1622796,2.0,3,1206,1286,61734.0
2749905,2749905,2.0,6,1960,3101,113038.0
370952,370952,2.0,2,1163,1286,218086.0
64842,64842,2.0,3,1540,3101,81170.0
